# 02 — Training Logs Analysis

Visualise training curves (loss, learning rate) from TensorBoard logs
and evaluate final model quality.

In [ ]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import numpy as np
import matplotlib.pyplot as plt

## 1. Load TensorBoard Event Files
Parse scalar summaries from log directory.

In [ ]:
from tensorflow.core.util import event_pb2
from tensorflow.python.lib.io import tf_record

log_dir = ROOT / 'logs' / 'tensorboard'

def read_scalars(logdir: Path):
    """Read scalar summaries from all event files under logdir."""
    data = {}
    for f in sorted(logdir.rglob('events.out.tfevents.*')):
        for raw in tf_record.tf_record_iterator(str(f)):
            event = event_pb2.Event.FromString(raw)
            for v in event.summary.value:
                data.setdefault(v.tag, []).append((event.step, v.simple_value))
    return data

if log_dir.exists():
    scalars = read_scalars(log_dir)
    print('Available tags:', list(scalars.keys()))
else:
    scalars = {}
    print('No logs found yet — run training first.')

## 2. Loss Curves

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

for tag, colour in [('loss/train', 'tab:blue'), ('loss/val', 'tab:orange')]:
    if tag in scalars:
        steps, vals = zip(*scalars[tag])
        ax.plot(steps, vals, label=tag, color=colour)

ax.set_xlabel('Epoch')
ax.set_ylabel('Balanced Logistic Loss')
ax.set_title('Training & Validation Loss')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Learning Rate Schedule

In [ ]:
if 'lr' in scalars:
    steps, vals = zip(*scalars['lr'])
    plt.figure(figsize=(8, 3))
    plt.plot(steps, vals, color='tab:green')
    plt.xlabel('Epoch')
    plt.ylabel('LR')
    plt.title('Learning Rate (cosine + warmup)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print('No LR data found.')

## 4. Quick Model Evaluation
Load the best checkpoint and compute response map quality on synthetic data.

In [ ]:
from src.training.siamese_model import build_inference_model

weights = ROOT / 'models' / 'siamese_tracker_tf' / 'best_weights.h5'
if weights.exists():
    model = build_inference_model(weights_path=weights)
    t = np.random.rand(4, 127, 127, 3).astype(np.float32)
    s = np.random.rand(4, 255, 255, 3).astype(np.float32)
    preds = model.predict([t, s], verbose=0)
    print(f'Prediction shape: {preds.shape}, mean score: {preds.mean():.4f}')
else:
    print('No trained weights found — run `make train` first.')